In [307]:
import sqlalchemy
import pandas as pd
from sqlalchemy.orm import sessionmaker
import requests
import json
from datetime import datetime
import datetime
import sqlite3

# DATABASE_LOCATION = 'my_played_tracks.sqlite'
USER_ID = '227dt3g18rrzyco2j7p2boy2ly'
TOKEN = 'BQBvzAThLG3FZTsRuaHcbzzFyxJnhd0Hkd3lXx-65bKs6Aop8_dVhDk0sgs08yyvhVAuajh-DKTHEYCb687f5D8Vl9EW8Req7q8d8pDRQQvATgtHSwZ6tPF7sZ95x76DR9xUFFdcSE3c73R0SQafed0Z6Vk1_5W4b252Sjwe'

In [308]:
headers = {
    "Accept":"application/json",
    "Content-Type":"application/json",
    "Authorization":"Bearer {token}".format(token=TOKEN)
}
    

In [309]:
r = requests.get("https://api.spotify.com/v1/me/player/recently-played",headers = headers)
response = r.json()

In [317]:
my_song_list = []

for song in response['items']:
    artist_id = song['track']['artists'][0]['id']
    artist_name = song['track']['artists'][0]['name']
    artist_link = song['track']['artists'][0]['external_urls']['spotify'] 
    album_id = song['track']['album']['id']
    album_name = song['track']['album']['name']
    album_link = song['track']['album']['external_urls']['spotify']
    song_id = song['track']['id']
    song_name = song['track']['name']
    song_link = song['track']['external_urls']['spotify']
    duration_ms = song['track']['duration_ms']
    popularity = song['track']['popularity']
    disc_number = song['track']['disc_number']
    played_at = song['played_at']
    
    song_dic = {'artist_id': artist_id,
                     'artist_name':artist_name,
                     'artist_link':artist_link,
                     'album_id':album_id,
                     'album_name':album_name,
                     'album_link':album_link,
                     'song_id':song_id,
                     'song_name':song_name,
                     'song_link':song_link,
                     'duration_ms':duration_ms,
                     'popularity':popularity,
                     'disc_number':disc_number,
                     'played_at':played_at
                     }
    
    my_song_list.append(song_dic)
    
    df = pd.DataFrame(song_dic, columns = ['artist_name','artist_link','album_id','album_name','album_link','song_id','song_name','song_link','duration_ms','popularity','disc_number','played_at'],index=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])

In [318]:
df

,artist_name,artist_link,album_id,album_name,album_link,song_id,song_name,song_link,duration_ms,popularity,disc_number,played_at
0,Dragnar,https://open.spotify.com/artist/35iYFweKkHPhUK...,7CJVXWuYVedL5curf9KeLF,Autumn Blush,https://open.spotify.com/album/7CJVXWuYVedL5cu...,6ignchvwPkXh6xjLQR5tCS,Autumn Blush,https://open.spotify.com/track/6ignchvwPkXh6xj...,262429,27,1,2021-12-21T21:12:13.579Z
1,Dragnar,https://open.spotify.com/artist/35iYFweKkHPhUK...,7CJVXWuYVedL5curf9KeLF,Autumn Blush,https://open.spotify.com/album/7CJVXWuYVedL5cu...,6ignchvwPkXh6xjLQR5tCS,Autumn Blush,https://open.spotify.com/track/6ignchvwPkXh6xj...,262429,27,1,2021-12-21T21:12:13.579Z
2,Dragnar,https://open.spotify.com/artist/35iYFweKkHPhUK...,7CJVXWuYVedL5curf9KeLF,Autumn Blush,https://open.spotify.com/album/7CJVXWuYVedL5cu...,6ignchvwPkXh6xjLQR5tCS,Autumn Blush,https://open.spotify.com/track/6ignchvwPkXh6xj...,262429,27,1,2021-12-21T21:12:13.579Z
3,Dragnar,https://open.spotify.com/artist/35iYFweKkHPhUK...,7CJVXWuYVedL5curf9KeLF,Autumn Blush,https://open.spotify.com/album/7CJVXWuYVedL5cu...,6ignchvwPkXh6xjLQR5tCS,Autumn Blush,https://open.spotify.com/track/6ignchvwPkXh6xj...,262429,27,1,2021-12-21T21:12:13.579Z
4,Dragnar,https://open.spotify.com/artist/35iYFweKkHPhUK...,7CJVXWuYVedL5curf9KeLF,Autumn Blush,https://open.spotify.com/album/7CJVXWuYVedL5cu...,6ignchvwPkXh6xjLQR5tCS,Autumn Blush,https://open.spotify.com/track/6ignchvwPkXh6xj...,262429,27,1,2021-12-21T21:12:13.579Z
5,Dragnar,https://open.spotify.com/artist/35iYFweKkHPhUK...,7CJVXWuYVedL5curf9KeLF,Autumn Blush,https://open.spotify.com/album/7CJVXWuYVedL5cu...,6ignchvwPkXh6xjLQR5tCS,Autumn Blush,https://open.spotify.com/track/6ignchvwPkXh6xj...,262429,27,1,2021-12-21T21:12:13.579Z
6,Dragnar,https://open.spotify.com/artist/35iYFweKkHPhUK...,7CJVXWuYVedL5curf9KeLF,Autumn Blush,https://open.spotify.com/album/7CJVXWuYVedL5cu...,6ignchvwPkXh6xjLQR5tCS,Autumn Blush,https://open.spotify.com/track/6ignchvwPkXh6xj...,262429,27,1,2021-12-21T21:12:13.579Z
7,Dragnar,https://open.spotify.com/artist/35iYFweKkHPhUK...,7CJVXWuYVedL5curf9KeLF,Autumn Blush,https://open.spotify.com/album/7CJVXWuYVedL5cu...,6ignchvwPkXh6xjLQR5tCS,Autumn Blush,https://open.spotify.com/track/6ignchvwPkXh6xj...,262429,27,1,2021-12-21T21:12:13.579Z
8,Dragnar,https://open.spotify.com/artist/35iYFweKkHPhUK...,7CJVXWuYVedL5curf9KeLF,Autumn Blush,https://open.spotify.com/album/7CJVXWuYVedL5cu...,6ignchvwPkXh6xjLQR5tCS,Autumn Blush,https://open.spotify.com/track/6ignchvwPkXh6xj...,262429,27,1,2021-12-21T21:12:13.579Z
9,Dragnar,https://open.spotify.com/artist/35iYFweKkHPhUK...,7CJVXWuYVedL5curf9KeLF,Autumn Blush,https://open.spotify.com/album/7CJVXWuYVedL5cu...,6ignchvwPkXh6xjLQR5tCS,Autumn Blush,https://open.spotify.com/track/6ignchvwPkXh6xj...,262429,27,1,2021-12-21T21:12:13.579Z
